In [1]:
# !apt-get install build-essential python3-dev
# !apt-get install cmake
# !pip install XFoil
!pip install torch torchvision

In [2]:
# code for generating airfoil designs using online airfoil datasets
# code is inspired by:
# The Department of Energy [https://catalog.data.gov/dataset/airfoil-computational-fluid-dynamics-2k-shapes-25-aoas-3-re-numbers]



# specify which platform we are running on 


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import sys
import math
import sklearn
import platform
import subprocess


# for the GAN
import torch
import torch.nn as nn

# importing xfoil itself
# from xfoil import XFoil
# from xfoil.model import Airfoil



# state variables
MAC_PLATFORM = 1
LINUX_PLATFORM = 2
UNKNOWN_PLATFORM = 3
PLATFORM = UNKNOWN_PLATFORM

In [3]:
# get the platform
os_type = platform.system()
if os_type == "Darwin":
	PLATFORM = MAC_PLATFORM
elif os_type == "Linux":
	PLATFORM = LINUX_PLATFORM
else:
	PLATFORM = UNKNOWN_PLATFORM

# should we be verbose
VERBOSE = False

# getting the file that we are currently working in 
# CURR_FILE_DIR = os.path.dirname(os.path.realpath(__file__))
CURR_FILE_DIR = "."

# define the directory that we are working in for the example files
EXAMPLE_DIR = "Example Data"

# setting the path of the airfoil
full_data_path = os.path.join(CURR_FILE_DIR, "airfoil_2k_data.h5")

# path to the example data
example_data_path = os.path.join(CURR_FILE_DIR, EXAMPLE_DIR, "b737a.dat")

# defining the data files that we are gonna use
output_file = "polar.dat"
example_data_output = os.path.join(CURR_FILE_DIR, EXAMPLE_DIR, output_file)
dump_file = "dump.dat"
example_data_dump = os.path.join(CURR_FILE_DIR, EXAMPLE_DIR, dump_file)


# the number of data points that describe each of the point clouds
NUM_POINTS_POINT_CLOUD = 100


# define the xfoil command depending on the platform
## CHANGE THIS IF NEEDED
if PLATFORM == MAC_PLATFORM:
	home_directory = os.path.expanduser("~")
	XFOIL_COMMAND = f"{home_directory}/Desktop/Xfoil-for-Mac/bin/xfoil"
elif PLATFORM == LINUX_PLATFORM:
	XFOIL_COMMAND = "xfoil"
else:
	raise NotImplementedError("Running program on unsupported platform.")

In [4]:
# now go ahead and define the gan that we are going to be training
class Generator(nn.Module):
    
    def __init__(self, dropout_p=0.2):
        
        super(Generator, self).__init__()
        
        self.fc = nn.Sequential(
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_p),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_p),
            nn.Linear(128, 100),
            # nn.LeakyReLU(0.2),
            # nn.Dropout(dropout_p),
            # nn.Linear(101, 256),  
        )

    def forward(self, noise, c_lift):
        x = torch.cat((noise, c_lift.unsqueeze(1)), 1)  
        return self.fc(x)

class Discriminator(nn.Module):
    def __init__(self):

        super(Discriminator, self).__init__()

        # defining the sequential discriminator
        self.fc = nn.Sequential(
            nn.Linear(101, 1024),  # 100 coordinates + 1 lift coefficient
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, points, c_lift):
        x = torch.cat((points, c_lift.unsqueeze(1)), 1)
        return self.fc(x)
    


In [5]:
# function for initializing the weights
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
        if hasattr(m, 'bias') and m.bias is not None:
            nn.init.constant_(m.bias.data, 0)

# create the models
generator = Generator()
discriminator = Discriminator()

# initialize the weights 
generator.apply(weights_init)
discriminator.apply(weights_init)

# create the loss function
criterion = nn.BCELoss()

# create optimizers for the generator and discriminator
optimizerG = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerD = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# function to create noise
def generate_noise(size):
    return torch.randn(size, NUM_POINTS_POINT_CLOUD)  

# training loop
def train_GAN(epochs, batch_size, data_loader):

    for epoch in range(epochs):

        for i, data in enumerate(data_loader, 0):

            # update D: maximize log(D(x)) + log(1 - D(G(z)))
            discriminator.zero_grad()

            # getting the data and info about the data
            real_data, labels = data
            batch_size = real_data.size(0)

            # getting random lift values for the batch
            real_c_lift = torch.FloatTensor(batch_size, 1).uniform_(0, 1)  

            # run the discriminator
            output = discriminator(real_data, real_c_lift).view(-1)

            # retrieving error
            errD_real = criterion(output, torch.ones(batch_size))
            errD_real.backward()
            D_x = output.mean().item()

            # generating noise for the generator
            noise = generate_noise(batch_size)
            fake_data = generator(noise, real_c_lift)

            # discriminator output
            output = discriminator(fake_data.detach(), real_c_lift).view(-1)

            # error propagation
            errD_fake = criterion(output, torch.zeros(batch_size))
            errD_fake.backward()
            D_G_z1 = output.mean().item()
            errD = errD_real + errD_fake
            optimizerD.step()

            # update generator: maximize log(D(G(z)))
            generator.zero_grad()
            output = discriminator(fake_data, real_c_lift).view(-1)
            errG = criterion(output, torch.ones(batch_size))
            errG.backward()
            D_G_z2 = output.mean().item()
            optimizerG.step()

            # update us with the epochs
            if i % 5 == 0:
                print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
                      % (epoch, epochs, i, len(data_loader), errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))


In [9]:
# getting the points from a file path
def get_points_from_dat_file(file_path):

	# open the file
	with open(file_path, 'r') as file:
		lines = file.readlines()

	# filter out non-numeric lines and strip whitespace
	points = []
	for line in lines:
		parts = line.strip().split()
		if len(parts) == 2:
			try:
				points.append((float(parts[0]), float(parts[1])))
			except ValueError:
				# print(f"Skipping invalid line: {line}")
				pass

	return points

# pull data tensor and information
def get_folder_data(folder_path = None, file_name = None):

	# pull the data tensor
	points = get_points_from_dat_file(os.path.join(folder_path, file_name + "_reformatted.dat"))
		
	# get the tensor
	points = torch.Tensor(points)

	# get the other parameters that we want
	resulting_data = pd.read_csv(os.path.join(folder_path, "polar.dat"), delim_whitespace=True, skiprows=10)
	resulting_data = resulting_data.drop(0)

	# pulling individual values
	alpha = resulting_data["alpha"].loc[0]
	CL = resulting_data['CL'].loc[0]         
	CD = resulting_data['CD'].loc[0]        
	CDp = resulting_data['CDp'].loc[0]         
	CM = resulting_data['CM'].loc[0]  

	# create labels tensor
	labels = torch.Tensor([CL, CDp, CM])

	# returning the generated values
	return (points, labels)
	
	
	

# define a function to create the tensors that we need
def pull_airfoil_data(wd=EXAMPLE_DIR):

	# the lists we will return
	ret_data_tensor = []
	ret_labels_tensor = []

	# iterate through all of the potential airfoils
	for airfoil_dat in os.listdir(wd):

		# print the airfoil type
		data_path = os.path.join(airfoil_dat)

		# pull the data
		try:
			t_data, t_labels = get_folder_data(folder_path=data_path, file_name=airfoil_dat)
		except:
			continue

		# add the data to the list
		ret_data_tensor.append(t_data)
		ret_labels_tensor.append(t_labels)

	# change the final versions to tensors
	ret_data_tensor = torch.Tensor(ret_data_tensor)
	ret_labels_tensor = torch.Tensor(ret_labels_tensor)

	return ret_data_tensor, ret_labels_tensor



from torch.utils.data import DataLoader, TensorDataset

# the data sets are going to be formed of:
# 100-dimensional vector representing the input airfoil

# 3-dimensional vector representing the lift coefficient, drag-pressure coefficient, and moment coefficient

airfoil_data, lift_values = pull_airfoil_data(wd=EXAMPLE_DIR)

# creating the datasets
dataset = TensorDataset(torch.tensor(airfoil_data, dtype=torch.float32), torch.tensor(lift_values, dtype=torch.float32))
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

train_GAN(epochs=50, batch_size=32, data_loader=dataloader)


tensor([])


/var/folders/d2/lsvbp7p92gnc5ql59sc72rx40000gn/T/ipykernel_68074/625747511.py:93: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset = TensorDataset(torch.tensor(airfoil_data, dtype=torch.float32), torch.tensor(lift_values, dtype=torch.float32))


ValueError: num_samples should be a positive integer value, but got num_samples=0